In [2]:
!pip install convokit
!pip install transformers
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install openai



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from convokit import Corpus, download
se_corpus = Corpus(filename=download("stack-exchange-politeness-corpus"))


Dataset already exists at /home/agyanesh/.convokit/downloads/stack-exchange-politeness-corpus


In [7]:
wiki_corpus = Corpus(filename=download("wikipedia-politeness-corpus"))


Dataset already exists at /home/agyanesh/.convokit/downloads/wikipedia-politeness-corpus


In [8]:
import pandas as pd

def extract_politeness_data(corpus):
    data = []
    for utt in corpus.iter_utterances():
        text = utt.text
        score = utt.meta['Normalized Score']
        data.append({'text': text, 'score': score})
    return pd.DataFrame(data)


In [9]:
se_df = extract_politeness_data(se_corpus)
wiki_df = extract_politeness_data(wiki_corpus)


In [10]:
def label_data(df):
    q1 = df['score'].quantile(0.25)
    q3 = df['score'].quantile(0.75)

    def label_score(score):
        if score <= q1:
            return 'impolite'
        elif score >= q3:
            return 'polite'
        else:
            return 'neutral'

    df['label'] = df['score'].apply(label_score)
    return df

se_df = label_data(se_df)
wiki_df = label_data(wiki_df)


In [11]:
def filter_data(df):
    return df[df['label'] != 'neutral'].reset_index(drop=True)

se_df_filtered = filter_data(se_df)
wiki_df_filtered = filter_data(wiki_df)


In [12]:
from sklearn.model_selection import train_test_split

def split_data(df):
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
    return train_df, test_df

se_train_df, se_test_df = split_data(se_df_filtered)
wiki_train_df, wiki_test_df = split_data(wiki_df_filtered)


In [13]:
from convokit import PolitenessStrategies

ps = PolitenessStrategies()

def extract_features(corpus, df):
    corpus = ps.transform(corpus)
    features = []
    labels = []
    for utt in corpus.iter_utterances():
        if utt.text in df['text'].values:
            features.append(utt.meta['politeness_strategies'])
            labels.append(df[df['text'] == utt.text]['label'].values[0])
    feature_df = pd.DataFrame(features)
    feature_df['label'] = labels
    return feature_df.fillna(0)


In [14]:
def update_corpus(corpus, df):
    texts = set(df['text'].values)
    utterances = [utt for utt in corpus.iter_utterances() if utt.text in texts]
    corpus_filtered = Corpus(utterances=utterances)
    return corpus_filtered


In [15]:
se_train_corpus = update_corpus(se_corpus, se_train_df)
se_test_corpus = update_corpus(se_corpus, se_test_df)

wiki_train_corpus = update_corpus(wiki_corpus, wiki_train_df)
wiki_test_corpus = update_corpus(wiki_corpus, wiki_test_df)

In [16]:
se_train_features = extract_features(se_train_corpus, se_train_df)
se_test_features = extract_features(se_test_corpus, se_test_df)

wiki_train_features = extract_features(wiki_train_corpus, wiki_train_df)
wiki_test_features = extract_features(wiki_test_corpus, wiki_test_df)


In [17]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

def train_svm(train_features):
    X_train = train_features.drop('label', axis=1)
    y_train = train_features['label']
    clf = LinearSVC()
    clf.fit(X_train, y_train)
    return clf


In [18]:
se_clf = train_svm(se_train_features)
wiki_clf = train_svm(wiki_train_features)

In [19]:
def evaluate_model(clf, test_features):
    X_test = test_features.drop('label', axis=1)
    y_test = test_features['label']
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

se_ling_accuracy = evaluate_model(se_clf, se_test_features)
wiki_ling_accuracy = evaluate_model(wiki_clf, wiki_test_features)

print(f"Stack Exchange Ling Model Accuracy: {se_ling_accuracy*100:.2f}%")
print(f"Wikipedia Ling Model Accuracy: {wiki_ling_accuracy*100:.2f}%")


Stack Exchange Ling Model Accuracy: 57.74%
Wikipedia Ling Model Accuracy: 73.58%


In [17]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/home/agyanesh/baseline_anlp/anlpb1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
from torch.utils.data import Dataset
import torch
class PolitenessDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
        self.labels = labels.apply(lambda x: 1 if x == 'polite' else 0).tolist()

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.labels)


In [19]:
se_train_dataset = PolitenessDataset(se_train_df['text'], se_train_df['label'])
se_test_dataset = PolitenessDataset(se_test_df['text'], se_test_df['label'])

wiki_train_dataset = PolitenessDataset(wiki_train_df['text'], wiki_train_df['label'])
wiki_test_dataset = PolitenessDataset(wiki_test_df['text'], wiki_test_df['label'])


In [20]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

def train_bert(train_dataset, eval_dataset):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        greater_is_better=True,
        save_total_limit=1,
        # The fix: Set save_strategy to "epoch" to match evaluation_strategy
        save_strategy="epoch",
    )

    def compute_metrics(p):
        preds = p.predictions.argmax(-1)
        labels = p.label_ids
        acc = (preds == labels).mean()
        return {'accuracy': acc}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer

In [21]:
print("Training BERT on Stack Exchange data:")
se_trainer = train_bert(se_train_dataset, se_test_dataset)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training BERT on Stack Exchange data:


/home/agyanesh/baseline_anlp/anlpb1/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.604400,0.595718,0.686838
2,0.432700,0.712424,0.639939
3,0.262200,0.871595,0.676248


In [22]:
print("Training BERT on Wikipedia data:")
wiki_trainer = train_bert(wiki_train_dataset, wiki_test_dataset)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training BERT on Wikipedia data:


/home/agyanesh/baseline_anlp/anlpb1/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.340200,0.337390,0.860092
2,0.105100,0.393795,0.885321
3,0.072200,0.467031,0.892202


In [23]:
def evaluate_bert(trainer, test_dataset):
    metrics = trainer.evaluate(eval_dataset=test_dataset)
    accuracy = metrics['eval_accuracy']
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return accuracy

se_bert_accuracy = evaluate_bert(se_trainer, se_test_dataset)
wiki_bert_accuracy = evaluate_bert(wiki_trainer, wiki_test_dataset)


Test Accuracy: 68.68%


Test Accuracy: 89.22%


In [1]:
!pip install openai==0.28


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import os
import openai


openai.api_key = 'sk-zdV6D5FqM-TU2j_PPQsJzA'

openai.api_base = 'https://cmu.litellm.ai'

# (Optional) Set the API type if required by the proxy
openai.api_type = 'open_ai'  # Adjust if necessary




In [28]:
def get_chatgpt_prediction(text):
    prompt = f"Do a binary classification on the politeness for the given text. If it's polite then output 1. Otherwise output 0 for impolite.\n\n{text}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model supported by the proxy
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1,
        temperature=0,
    )

    prediction = response['choices'][0]['message']['content'].strip()
    return prediction


In [29]:
import time

def get_predictions(df):
    predictions = []
    for idx, row in df.iterrows():
        text = row['text']
        prediction = get_chatgpt_prediction(text)
        predictions.append(prediction)
        time.sleep(1)  # Adjust based on rate limits
    return predictions


In [30]:
se_chatgpt_predictions_raw = get_predictions(se_test_df)
wiki_chatgpt_predictions_raw = get_predictions(wiki_test_df)


In [31]:
def process_predictions(predictions):
    processed = []
    for pred in predictions:
        if '1' in pred:
            processed.append('polite')
        elif '0' in pred:
            processed.append('impolite')
        else:
            processed.append('unknown')
    return processed

se_chatgpt_predictions = process_predictions(se_chatgpt_predictions_raw)
wiki_chatgpt_predictions = process_predictions(wiki_chatgpt_predictions_raw)

def evaluate_chatgpt(df, predictions):
    df = df.copy()
    df['prediction'] = predictions
    df = df[df['prediction'] != 'unknown']
    accuracy = (df['label'] == df['prediction']).mean()
    return accuracy

se_chatgpt_accuracy = evaluate_chatgpt(se_test_df, se_chatgpt_predictions)
wiki_chatgpt_accuracy = evaluate_chatgpt(wiki_test_df, wiki_chatgpt_predictions)

print(f"Stack Exchange ChatGPT Accuracy: {se_chatgpt_accuracy*100:.2f}%")
print(f"Wikipedia ChatGPT Accuracy: {wiki_chatgpt_accuracy*100:.2f}%")


Stack Exchange ChatGPT Accuracy: 59.76%
Wikipedia ChatGPT Accuracy: 73.96%
